In [1]:
# ruff: noqa: F401, E402
import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from binary_index_tree import BinIndexTree
from combinatorics import fib
from parsing import format_binop_expression, parse_binop_expression
from sequences import find_if
from strings import StringView, splint

print()


In [48]:
def partition_by_sum(a: list, start: int = 0, stop=None) -> int:
    if stop is None:
        stop = len(a)
    csum = list(accumulate(a, initial=0))
    l, h = start + 1, stop - 1
    mn = (csum[stop] - csum[start], csum[start])
    while l <= h:
        m = (l + h) // 2
        ls = csum[m] - csum[start]
        rs = csum[stop] - csum[m]
        if ls < rs:
            mn = min(mn, (rs - ls, ls))
            l = m + 1
        else:
            mn = min(mn, (ls - rs, rs))
            h = m - 1
    return mn[1], sum(mn)


In [49]:
partition_equal_sum([2, 2, 2, 2])


(4, 4)

In [50]:
partition_equal_sum([2])


(0, 2)

In [51]:
partition_equal_sum([])


(0, 0)

In [ ]:
four_set_min_diff([16, 8, 11, 2, 12, 4, 11, 16, 15, 2, 5, 18, 7, 17])
